In [1]:
import pandas as pd
import numpy as np

In [51]:
df = pd.read_csv('data/fma_dataset.csv')
df.head()

,track_id,title_track,date_released,title_album,artist,duration,genre,favorites_artist,listens_track,favorites_track,listens_album,favorites_album
0,2,Food,2009-01-05 00:00:00,AWOL - A Way Of Life,AWOL,168,Hip-Hop,9,1293,2,6073,4
1,3,Electric Ave,2009-01-05 00:00:00,AWOL - A Way Of Life,AWOL,237,Hip-Hop,9,514,1,6073,4
2,5,This World,2009-01-05 00:00:00,AWOL - A Way Of Life,AWOL,206,Hip-Hop,9,1151,6,6073,4
3,10,Freeway,2008-02-06 00:00:00,Constant Hitmaker,Kurt Vile,161,Pop,74,50135,178,47632,4
4,20,Spiritual Level,2009-01-06 00:00:00,Niris,Nicky Cook,311,NaN,10,361,0,2710,2


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106574 entries, 0 to 106573
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   track_id          106574 non-null  int64 
 1   title_track       106573 non-null  object
 2   date_released     70294 non-null   object
 3   title_album       105549 non-null  object
 4   artist            106574 non-null  object
 5   duration          106574 non-null  int64 
 6   genre             49598 non-null   object
 7   favorites_artist  106574 non-null  int64 
 8   listens_track     106574 non-null  int64 
 9   favorites_track   106574 non-null  int64 
 10  listens_album     106574 non-null  int64 
 11  favorites_album   106574 non-null  int64 
dtypes: int64(7), object(5)
memory usage: 9.8+ MB


### Replace Null values and remove commas and partition date column

In [76]:
df.loc[df.title_album.isna(), 'title_album'] = '-'
df.loc[df.title_track.isna(), 'title_track'] = '-'
df.loc[df.date_released.isna(), 'date_released'] = '0000-00-00 00:00:00'
df.loc[df.genre.isna(), 'genre'] = '-'

df = df.replace(r',','', regex=True)
df = df.replace(r'\\','-', regex=True)
df = df.replace(r'/','-', regex=True)
df = df.replace(r'\'','', regex=True)
df = df.replace(r'\"','', regex=True)

df['year'] = df['date_released'].apply(lambda x: x[:4]).astype(int)
df['month'] = df['date_released'].apply(lambda x: x[5:7]).astype(int)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106574 entries, 0 to 106573
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   track_id          106574 non-null  int64 
 1   title_track       106574 non-null  object
 2   date_released     106574 non-null  object
 3   title_album       106574 non-null  object
 4   artist            106574 non-null  object
 5   duration          106574 non-null  int64 
 6   genre             106574 non-null  object
 7   favorites_artist  106574 non-null  int64 
 8   listens_track     106574 non-null  int64 
 9   favorites_track   106574 non-null  int64 
 10  listens_album     106574 non-null  int64 
 11  favorites_album   106574 non-null  int64 
 12  year              106574 non-null  int32 
 13  month             106574 non-null  int32 
dtypes: int32(2), int64(7), object(5)
memory usage: 10.6+ MB


## Q1
for each question only needed columns are saved and imported to cassandra

In [42]:
t1 = df[['title_album', 'track_id', 'title_track']]
t1.to_csv('data/t1.csv',index=False, header=False)
t1.head()

,title_album,track_id,title_track
0,AWOL - A Way Of Life,2,Food
1,AWOL - A Way Of Life,3,Electric Ave
2,AWOL - A Way Of Life,5,This World
3,Constant Hitmaker,10,Freeway
4,Niris,20,Spiritual Level


## Q2

In [88]:
t2 = df[['artist', 'track_id', 'title_track', 'genre']]
t2.to_csv('data/t2.csv', index=False, header=False)
t2.head()

,artist,track_id,title_track,genre
0,AWOL,2,Food,Hip-Hop
1,AWOL,3,Electric Ave,Hip-Hop
2,AWOL,5,This World,Hip-Hop
3,Kurt Vile,10,Freeway,Pop
4,Nicky Cook,20,Spiritual Level,-


## Q3 
change duration to a boolean. True for less than 3 minutes, False for more than 3 minutes

In [87]:
t3 = df[['genre', 'duration', 'year', 'track_id', 'title_track']]
t3.loc[:, 'duration'] = t3['duration'] < 180
# Mix genre and year together 
t3.to_csv('data/t3.csv', index=False, header=False)
t3.head()

C:\Users\yaram\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,genre,duration,year,track_id,title_track
0,Hip-Hop,True,2009,2,Food
1,Hip-Hop,False,2009,3,Electric Ave
2,Hip-Hop,False,2009,5,This World
3,Pop,True,2008,10,Freeway
4,-,False,2009,20,Spiritual Level


## Q4
change listens_track to a boolean. True for more than 300 views, False for less than 300 views

In [89]:
t4 = df[['month', 'listens_track', 'genre', 'track_id', 'title_track']]
t4.loc[:, 'listens_track'] = t4['listens_track'] >= 300
# Mix genre and year together 
t4.to_csv('data/t4.csv', index=False, header=False)
t4.head()

C:\Users\yaram\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,month,listens_track,genre,track_id,title_track
0,1,True,Hip-Hop,2,Food
1,1,True,Hip-Hop,3,Electric Ave
2,1,True,Hip-Hop,5,This World
3,2,True,Pop,10,Freeway
4,1,True,-,20,Spiritual Level


In [85]:
t4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106574 entries, 0 to 106573
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   month          106574 non-null  int32 
 1   listens_track  106574 non-null  bool  
 2   genre          106574 non-null  object
 3   track_id       106574 non-null  int64 
 4   title_track    106574 non-null  object
dtypes: bool(1), int32(1), int64(1), object(2)
memory usage: 2.9+ MB


# Q6
q5 did not exist

In [90]:
t6 = df[['genre', 'year', 'month', 'track_id']]
t6.to_csv('data/t6.csv', index=False, header=False)
t6.head()

,genre,year,month,track_id
0,Hip-Hop,2009,1,2
1,Hip-Hop,2009,1,3
2,Hip-Hop,2009,1,5
3,Pop,2008,2,10
4,-,2009,1,20


## Q7

In [92]:
t7 = df[['duration', 'track_id', 'title_track']]
# Add constant partition key! I want all my data to be in one partition
t7['partition'] = 1
t7.to_csv('data/t7.csv', index=False, header=False)
t7.head()

<ipython-input-92-c540733646a7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t7['partition'] = 1


,duration,track_id,title_track,partition
0,168,2,Food,1
1,237,3,Electric Ave,1
2,206,5,This World,1
3,161,10,Freeway,1
4,311,20,Spiritual Level,1


## Q8

In [98]:
# No need to save track_id, because favorites_album just repeats for all songs
# of an album and we just need one of them
t8 = df[['genre', 'year', 'favorites_album', 'favorites_artist', 'title_album']]
t8.to_csv('data/t8.csv', index=False, header=False)
t8.head()

,genre,year,favorites_album,favorites_artist,title_album
0,Hip-Hop,2009,4,9,AWOL - A Way Of Life
1,Hip-Hop,2009,4,9,AWOL - A Way Of Life
2,Hip-Hop,2009,4,9,AWOL - A Way Of Life
3,Pop,2008,4,74,Constant Hitmaker
4,-,2009,2,10,Niris


## Q9

In [99]:
t9 = df[['genre', 'listens_track']]
# Add constant partition key! I want all my data to be in one partition
t9['partition'] = 1
t9.to_csv('data/t9.csv', index=False, header=False)
t9.head()


<ipython-input-99-9a42c70d1352>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t9['partition'] = 1


,genre,listens_track,partition
0,Hip-Hop,1293,1
1,Hip-Hop,514,1
2,Hip-Hop,1151,1
3,Pop,50135,1
4,-,361,1


## Q10

In [100]:
t9 = df[['year', 'listens_album', 'artist']]
t9.to_csv('data/10.csv', index=False, header=False)
t9.head()

,year,listens_album,artist
0,2009,6073,AWOL
1,2009,6073,AWOL
2,2009,6073,AWOL
3,2008,47632,Kurt Vile
4,2009,2710,Nicky Cook
